In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
donors = ScrapedContribution.objects.filter(
    Q(filing_committee__committee_group='leg', filing_committee__party='D') |
    Q(filing_committee__name__in=[
        'HARRY TRUMAN FUND', 'HOUSE DEMO CAMP COMM', '7TH LEG DIST DEMO CENT COMM NON EXEMPT',
        'WA ST DEMO CENT COMM EXEMPT', 'WA ST DEMO CENT COMM NON EXEMPT', 'WA ST PROGRESSIVE CAUCUS',
        'WA SENATE DEMO CAMP', 'KENNEDY FUND']))

seiu = donors.filter(Q(donor__icontains='seiu') | Q(donor__icontains='service employee'))
print seiu.aggregate(Sum('amount'))

donors = donors.values(
    'donor', 'city', 'state').annotate(sum=Sum('amount'), num=Count('donor')).order_by('-sum')
refunds = ScrapedRefund.objects.filter(filing_committee__committee_group='leg').values(
    'donor', 'city', 'state').annotate(sum=Sum('amount'), num=Count('donor'))
data = {}
for donor in donors:
    key = '-'.join([donor['donor'], donor['city'], donor['state']])
    data[key] = {
        'total': donor['sum'],
        'num': donor['num'],
        'total_adjusted': donor['sum'],
        'num_refunds': 0
    }
for donor in refunds:
    key = '-'.join([donor['donor'], donor['city'], donor['state']])
    if key in data:
        data[key]['total_adjusted'] -= donor['sum']
        data[key]['num_refunds'] += donor['num']

for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_adjusted'], reverse=True)[:500]):
    print '%d) %s: %s in %s contribs, after adjusting for %d refunds' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_adjusted'], grouping=True),
        locale.format('%d', data[group]['num'], grouping=True),
        data[group]['num_refunds'])

{'amount__sum': Decimal('1668825.000')}
1) HOUSE DEMOCRATIC CAUCUS CAMPAIGN COMM.-SEATTLE-WA: $695,000.00 in 2 contribs, after adjusting for 0 refunds
2) WASHINGTON SENATE DEMOCRATIC CAMPAIGN-SEATTLE-WA: $599,000.00 in 8 contribs, after adjusting for 0 refunds
3) HOUSE DEMOCRATIC CAMPAIGN COMMITTEE-SEATTLE-WA: $447,500.00 in 21 contribs, after adjusting for 0 refunds
4) PUYALLUP TRIBE OF INDIANS-TACOMA-WA: $397,800.00 in 100 contribs, after adjusting for 3 refunds
5) SEIU WA STATE COUNCIL-SEATTLE-WA: $370,000.00 in 4 contribs, after adjusting for 0 refunds
6) WA EDUCATION ASSN PAC / WEA PAC-FEDERAL WAY-WA: $310,000.00 in 3 contribs, after adjusting for 0 refunds
7) DEMOCRATIC GOVERNORS ASSOCIATION-WASHINGTON-DC: $300,000.00 in 2 contribs, after adjusting for 0 refunds
8) ORR JAMES L-LAKEWOOD-WA: $252,000.00 in 3 contribs, after adjusting for 0 refunds
9) WASHINGTON FEDERATION OF STATE EMPLOYEES  (WFSE)-OLYMPIA-WA: $250,000.00 in 4 contribs, after adjusting for 0 refunds
10) BLOOMBERG M

In [3]:
for group in ScrapedContribution.objects.filter(filing_committee__committee_group='federal').values(
        'filing_committee__name').annotate(sum=Sum('amount')).order_by('-sum'):
    print group

{'filing_committee__name': u'MURRAY, PATTY', 'sum': Decimal('8170930.740')}
{'filing_committee__name': u'CLINTON, HILLARY RODHAM / KAINE, TIMOT', 'sum': Decimal('7069181.290')}
{'filing_committee__name': u'KILMER, DEREK', 'sum': Decimal('2149221.040')}
{'filing_committee__name': u'HECK, DENNIS', 'sum': Decimal('1531391.050')}
{'filing_committee__name': u'MCMORRIS RODGERS, CATHY', 'sum': Decimal('1460804.000')}
{'filing_committee__name': u'REICHERT, DAVE', 'sum': Decimal('1366703.930')}
{'filing_committee__name': u'JAYAPAL, PRAMILA', 'sum': Decimal('1273190.170')}
{'filing_committee__name': u'DELBENE, SUZAN K', 'sum': Decimal('866223.710')}
{'filing_committee__name': u'WALKINSHAW, BRADY PINERO', 'sum': Decimal('814685.790')}
{'filing_committee__name': u'SMITH, D ADAM', 'sum': Decimal('784258.460')}
{'filing_committee__name': u'HERRERA BEUTLER, JAIME', 'sum': Decimal('589610.200')}
{'filing_committee__name': u'MCDERMOTT, JAMES JOSEPH', 'sum': Decimal('429586.180')}
{'filing_committee__na

In [4]:
donors = ScrapedContribution.objects.filter(donor='JUSTICE FOR ALL PAC').values(
    'filing_committee__name').annotate(sum=Sum('amount'), num=Count('date')).order_by('-sum')
for d in donors:
    print '%s to %s in %d contribs' % (
        locale.currency(d['sum'], grouping=True), d['filing_committee__name'], d['num'])

$175,000.00 to OUR WASHINGTON in 6 contribs
$120,000.00 to HARRY TRUMAN FUND in 7 contribs
$115,000.00 to KNOW YOUR JUDICIAL BALLOT PAC in 1 contribs
$75,000.00 to KENNEDY FUND in 5 contribs
$40,000.00 to FUSE VOTES in 1 contribs
$27,500.00 to WA ST DEMO CENT COMM EXEMPT in 6 contribs
$25,000.00 to MYVOTECOUNTS in 1 contribs
$5,000.00 to 30TH DIST DEMO ORG EXEMPT in 1 contribs
$5,000.00 to 35TH LEG DIST DEMO COMM POLITICAL ACCT in 1 contribs
$5,000.00 to 26TH LEG DIST DEMO CENT COMM NON EXEMPT in 1 contribs
$5,000.00 to 31ST LEG DIST DEMO NON EXEMPT in 1 contribs
$5,000.00 to 17TH LEG DIST DEMO CENT COMM NON EXEMPT in 1 contribs
$5,000.00 to WA ST DEMO CENT COMM NON EXEMPT in 1 contribs
$4,000.00 to INSLEE JAY R in 5 contribs
$4,000.00 to MADSEN BARBARA A in 3 contribs
$4,000.00 to KREIDLER MYRON MIKE B in 3 contribs
$4,000.00 to FERGUSON ROBERT W in 7 contribs
$3,000.00 to DUNSHEE HANS M in 5 contribs
$3,000.00 to MILOSCIA MARK A in 2 contribs
$3,000.00 to PODLODOWSKI TINA M in 3 cont

In [5]:
donors = ScrapedContribution.objects.filter(donor__icontains='state dental pac', filing_committee__committee_group='leg').values(
    'filing_committee__name').annotate(sum=Sum('amount'), num=Count('date')).order_by('-sum')
print donors.count()
for d in donors:
    print '%s to %s in %d contribs' % (
        locale.currency(d['sum'], grouping=True), d['filing_committee__name'], d['num'])

101
$3,900.00 to VAN DE WEGE KEVIN W in 5 contribs
$2,950.00 to HAWKINS BRADLEY M in 3 contribs
$2,500.00 to FEY JACOB C in 3 contribs
$2,000.00 to MCBRIDE JOAN L in 2 contribs
$2,000.00 to SCHMICK JOSEPH S in 3 contribs
$2,000.00 to DEBOLT RICHARD C in 4 contribs
$2,000.00 to HAYES DAVID J in 3 contribs
$2,000.00 to NEALEY TERRY R in 2 contribs
$2,000.00 to ORTIZ-SELF LILLIAN in 2 contribs
$2,000.00 to JOHNSON NORMAN M in 3 contribs
$2,000.00 to CALDIER MICHELLE D in 4 contribs
$2,000.00 to MCCABE GINA R in 3 contribs
$2,000.00 to ROSSETTI JUSTIN D in 2 contribs
$2,000.00 to BARKIS ANDREW K in 2 contribs
$2,000.00 to CARLYLE REUVEN M in 2 contribs
$2,000.00 to HARMSWORTH MARK J in 2 contribs
$2,000.00 to HARGROVE MARK D in 3 contribs
$2,000.00 to KIRBY STEVEN T in 3 contribs
$2,000.00 to MULLET MARK D in 3 contribs
$2,000.00 to BECKER RANDI L in 3 contribs
$2,000.00 to PEARSON KIRK J in 2 contribs
$2,000.00 to ORCUTT EDMUND T in 3 contribs
$2,000.00 to MACEWEN DREW C in 2 contribs
$2,

In [6]:
100. * (88850./(88850.+53350.+1000.))

62.04608938547486

In [7]:
contribs = ScrapedContribution.objects.filter(filing_committee__name__icontains='DAMMEIER', donor__icontains='print').order_by('-amount')
for contrib in contribs:
    print contrib.donor, contrib.amount

In [8]:
expenses = ScrapedExpense.objects.filter(filing_committee__name__icontains='DAMMEIER', vendor__icontains='print').values('vendor').annotate(sum=Sum('amount'))
print expenses

[{'sum': Decimal('137453.130'), 'vendor': u'PRINT NW'}]
